In [54]:
import pandas as pd
import numpy as np


btradron = pd.read_csv("./.data/BTRADRON.csv", skiprows=16)

btradron["Currency"] = "RON"

btradeur = pd.read_csv("./.data/BTRADEUR.csv", skiprows=16)

btradeur["Currency"] = "EUR"


df = pd.concat([btradron, btradeur], ignore_index=True)

df.head()

,Data tranzactie,Data valuta,Descriere,Referinta tranzactiei,Debit,Credit,Sold contabil,Currency
0,2024-08-02,2024-08-02,Plata la POS non-BT cu card VISA;EPOS 30/07/20...,000NVPO242154Boq,"-1,043.83",NaN,164.30,RON
1,2024-08-01,2024-08-01,Pachet IZI;Nelimitat in reteaua BT;REF: E41IZ5...,E41IZ56241800002,-29.00,NaN,"1,208.13",RON
2,2024-07-31,2024-07-31,Plata la POS non-BT cu card VISA;EPOS 28/07/20...,000NVPO242134gxY,-36.99,NaN,"1,237.13",RON
3,2024-07-22,2024-07-22,Plata OP inter - canal electronic;/ROC/./RFB/2...,E41ZEXA2420400GK,"-1,301.00",NaN,535.52,RON
4,2024-07-22,2024-07-22,Comision plata OP;/ROC/./RFB/20240722/20240722...,E41ZEXA2420400GK,-5.00,NaN,"1,836.52",RON


In [55]:
df = df.drop(
    ["Referinta tranzactiei", "Credit", "Sold contabil", "Data valuta"], axis=1
)

df.head()

,Data tranzactie,Descriere,Debit,Currency
0,2024-08-02,Plata la POS non-BT cu card VISA;EPOS 30/07/20...,"-1,043.83",RON
1,2024-08-01,Pachet IZI;Nelimitat in reteaua BT;REF: E41IZ5...,-29.00,RON
2,2024-07-31,Plata la POS non-BT cu card VISA;EPOS 28/07/20...,-36.99,RON
3,2024-07-22,Plata OP inter - canal electronic;/ROC/./RFB/2...,"-1,301.00",RON
4,2024-07-22,Comision plata OP;/ROC/./RFB/20240722/20240722...,-5.00,RON


In [56]:
df["Payment Date"] = df["Descriere"].str.extract(r"(POS\s(\d{2}\/\d{2}\/\d{4}))")[1]
regex_pattern = r"Schimb valutar|LT383250069969855031|Comision"

mask = ~df["Descriere"].str.contains(regex_pattern)


filtered_df = df.loc[mask].copy()


date_formats = ["%d/%m/%Y", "%Y-%m-%d", "%d-%m-%Y"]  # Add all expected formats
for fmt in date_formats:
    filtered_df["Payment Date"] = pd.to_datetime(
        filtered_df["Payment Date"], format=fmt, errors="coerce"
    )


filtered_df["Data tranzactie"] = pd.to_datetime(df["Data tranzactie"], errors="coerce")

# Formatting dates to 'day-month-year'
filtered_df["Payment Date"] = filtered_df["Payment Date"].dt.strftime("%d-%m-%Y")
filtered_df["Data tranzactie"] = filtered_df["Data tranzactie"].dt.strftime("%d-%m-%Y")


filtered_df["Payment Date"] = filtered_df["Payment Date"].fillna(
    filtered_df["Data tranzactie"]
)


filtered_df.head()

,Data tranzactie,Descriere,Debit,Currency,Payment Date
0,02-08-2024,Plata la POS non-BT cu card VISA;EPOS 30/07/20...,"-1,043.83",RON,30-07-2024
1,01-08-2024,Pachet IZI;Nelimitat in reteaua BT;REF: E41IZ5...,-29.00,RON,01-08-2024
2,31-07-2024,Plata la POS non-BT cu card VISA;EPOS 28/07/20...,-36.99,RON,28-07-2024
3,22-07-2024,Plata OP inter - canal electronic;/ROC/./RFB/2...,"-1,301.00",RON,22-07-2024
5,07-07-2024,Plata la POS non-BT cu card VISA;EPOS 05/07/20...,-49.99,RON,05-07-2024


In [57]:
filtered_df["Merchant"] = filtered_df["Descriere"].str.extract(
    r"TID:\s*(?:[^\s]{2,8}\s+)?(\w+)"
)
filtered_df["Merchant"] = filtered_df["Merchant"].fillna(
    df["Descriere"].str.split(";").str[3]
)

In [58]:
filtered_df['Payment Date'] = pd.to_datetime(filtered_df['Payment Date'], errors='coerce', dayfirst=True)
filtered_df = filtered_df.sort_values(by="Payment Date", ascending=True)
filtered_df

,Data tranzactie,Descriere,Debit,Currency,Payment Date,Merchant
5,07-07-2024,Plata la POS non-BT cu card VISA;EPOS 05/07/20...,-49.99,RON,2024-07-05,APPLE
49,10-07-2024,Plata la POS non-BT cu card MASTERCARD;EPOS 08...,-33.05,EUR,2024-07-08,LINKEDIN
46,12-07-2024,Plata la POS non-BT cu card MASTERCARD;EPOS 10...,-12.27,EUR,2024-07-10,Amazon
45,14-07-2024,Plata la POS non-BT cu card MASTERCARD;EPOS 12...,-11.07,EUR,2024-07-12,Amazon
42,17-07-2024,Plata la POS non-BT cu card MASTERCARD;EPOS 15...,-39.80,EUR,2024-07-15,NESPRESSO
41,18-07-2024,Plata la POS non-BT cu card MASTERCARD;POS 16/...,-15.00,EUR,2024-07-16,Krefel
40,19-07-2024,Plata la POS non-BT cu card MASTERCARD;EPOS 17...,-14.91,EUR,2024-07-17,AMZN
39,19-07-2024,Plata la POS non-BT cu card MASTERCARD;POS 17/...,-30.02,EUR,2024-07-17,Q8
36,20-07-2024,Plata la POS non-BT cu card MASTERCARD;EPOS 18...,-42.00,EUR,2024-07-18,AMZN
38,20-07-2024,Retragere de numerar de la ATM non-BT cu card ...,-20.00,EUR,2024-07-20,JAMBES


In [60]:
new_column_order = ["Payment Date", "Merchant", "Descriere", "Debit", "Data tranzactie", "Currency"]

filtered_df = filtered_df[new_column_order]

filtered_df["Payment Date"] = pd.to_datetime(df["Payment Date"], errors="coerce")

filtered_df

/tmp/ipykernel_84900/3861002688.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  filtered_df["Payment Date"] = pd.to_datetime(df["Payment Date"], errors="coerce")


,Payment Date,Merchant,Descriere,Debit,Data tranzactie,Currency
5,2024-07-05,APPLE,Plata la POS non-BT cu card VISA;EPOS 05/07/20...,-49.99,07-07-2024,RON
49,2024-07-08,LINKEDIN,Plata la POS non-BT cu card MASTERCARD;EPOS 08...,-33.05,10-07-2024,EUR
46,2024-07-10,Amazon,Plata la POS non-BT cu card MASTERCARD;EPOS 10...,-12.27,12-07-2024,EUR
45,2024-07-12,Amazon,Plata la POS non-BT cu card MASTERCARD;EPOS 12...,-11.07,14-07-2024,EUR
42,2024-07-15,NESPRESSO,Plata la POS non-BT cu card MASTERCARD;EPOS 15...,-39.80,17-07-2024,EUR
41,2024-07-16,Krefel,Plata la POS non-BT cu card MASTERCARD;POS 16/...,-15.00,18-07-2024,EUR
40,2024-07-17,AMZN,Plata la POS non-BT cu card MASTERCARD;EPOS 17...,-14.91,19-07-2024,EUR
39,2024-07-17,Q8,Plata la POS non-BT cu card MASTERCARD;POS 17/...,-30.02,19-07-2024,EUR
36,2024-07-18,AMZN,Plata la POS non-BT cu card MASTERCARD;EPOS 18...,-42.00,20-07-2024,EUR
38,NaT,JAMBES,Retragere de numerar de la ATM non-BT cu card ...,-20.00,20-07-2024,EUR
